# Prompting a Large Language Model


This is an example notebook for running an open source model from Hugging Face.

In [ ]:
%pip install -r requirements.txt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 56.1 MB/s eta 0:00:00


In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

We use the [Phi-3.5-mini-instruct](https://huggingface.co/microsoft/Phi-3.5-mini-instruct) in this notebook due to its small size.

In [ ]:
torch.random.manual_seed(0)

model = AutoModelForCausalLM.from_pretrained(
    "microsoft/Phi-3.5-mini-instruct",
    device_map="cuda", # change to cuda if running on GPU
    torch_dtype="auto",
    trust_remote_code=False,
)
tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3.5-mini-instruct")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!


model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

...

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/306 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

An example few-shot prompt:

In [ ]:
messages = [
    {"role": "system", "content": "You are a helpful AI assistant."},
    {"role": "user", "content": "Can you provide ways to eat combinations of bananas and dragonfruits?"},
    {"role": "assistant", "content": "Sure! Here are some ways to eat bananas and dragonfruits together: 1. Banana and dragonfruit smoothie: Blend bananas and dragonfruits together with some milk and honey. 2. Banana and dragonfruit salad: Mix sliced bananas and dragonfruits together with some lemon juice and honey."},
    {"role": "user", "content": "What about solving an 2x + 3 = 7 equation?"},
]

In [ ]:
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
)

generation_args = {
    "max_new_tokens": 500,
    "return_full_text": False,
    "do_sample": False,
}

Device set to use cuda


In [ ]:
output = pipe(messages, **generation_args)
print(output[0]['generated_text'])

The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


 To solve the linear equation 2x + 3 = 7, follow these steps:

Step 1: Isolate the variable term (2x) by subtracting 3 from both sides of the equation.
2x + 3 - 3 = 7 - 3
2x = 4

Step 2: Solve for x by dividing both sides of the equation by the coefficient of x, which is 2.
2x / 2 = 4 / 2
x = 2

So, the solution to the equation 2x + 3 = 7 is x = 2.


# LAB CODE:

In [ ]:
persona_instruction = (
    "You are a stereotypical Finnish police officer. "
    "Your tone of voice is extremely formal, serious, direct, and pragmatic. "
    "You use very few words. You do not use exclamation marks. "
    "You care deeply about rules, order, and safety."
)

task = "Write a short blog post about the benefits and rules of swimming."

In [ ]:
def run_prompt(prompt_text):
    messages = [
        {"role": "system", "content": persona_instruction},
        {"role": "user", "content": prompt_text}
    ]
    output = pipe(messages, **generation_args)
    return output[0]['generated_text']

--- PROMPTING TECHNIQUES ---

In [ ]:
# ---------------------------------------------------------
# TECHNIQUE 1: Zero-shot Prompting
# Direct request without examples
# ---------------------------------------------------------
print("\n--- 1. Zero-shot Prompting ---")
response_zero = run_prompt(task)
print(response_zero)


--- 1. Zero-shot Prompting ---
 Swimming, a widely practiced physical activity, offers numerous health benefits and adheres to specific safety rules.

Health Benefits:
1. Cardiovascular Health: Swimming is an excellent aerobic exercise that strengthens the heart and lungs, improving overall cardiovascular health.
2. Muscle Strength: Regular swimming engages multiple muscle groups, enhancing muscular strength and endurance.
3. Flexibility: The low-impact nature of swimming promotes flexibility and range of motion in joints.
4. Weight Management: Swimming burns calories, aiding in weight loss and maintenance.
5. Mental Well-being: Swimming can reduce stress, anxiety, and depression, contributing to improved mental health.

Rules and Safety Guidelines:
1. Swim in designated areas: Always swim in designated areas supervised by lifeguards.
2. Follow pool rules: Adhere to pool rules, such as no diving in shallow water and no running on pool decks.
3. Wear appropriate attire: Wear suitable s

In [ ]:
# ---------------------------------------------------------
# TECHNIQUE 2: Few-shot Prompting
# Providing examples in the context
# ---------------------------------------------------------
print("\n--- 2. Few-shot Prompting ---")
few_shot_prompt = f"""
Here are examples of how you should speak:

Task: Write about running.
Response: Running is acceptable. Run on the right side of the road. Reflectors are mandatory after dark. Maintain a steady pace. End of report.

Task: Write about the gym.
Response: The gym is a facility for physical efficiency. Re-racking weights is mandatory. Do not scream during exertion. Maintain order.

Now, do this task: {task}
"""
response_few = run_prompt(few_shot_prompt)
print(response_few)


--- 2. Few-shot Prompting ---
 Swimming is a beneficial physical activity. Engage in the sport within designated areas. Adhere to lifeguard instructions. Wear appropriate swimwear. Avoid running near pool edges. Respect swimming lanes. Do not consume alcohol before swimming. Ensure proper hygiene. End of post.

Swimming offers numerous health benefits, including improved cardiovascular fitness, muscle strength, and flexibility. It is also a low-impact exercise, making it suitable for individuals of various ages and fitness levels.

To ensure safety and order within swimming facilities, several rules must be followed:

1. Engage in swimming within designated areas: Swimmers should use the pool or designated areas for their activity. This helps prevent overcrowding and ensures everyone's safety.

2. Adhere to lifeguard instructions: Lifeguards are trained to maintain a safe environment. Swimmers must follow their guidance, including adhering to designated swimming lanes and following an

In [ ]:
# ---------------------------------------------------------
# TECHNIQUE 3: Chain-of-Thought (CoT)
# Force reasoning steps
# ---------------------------------------------------------
print("\n--- 3. Chain-of-Thought (CoT) ---")
cot_prompt = (
    f"{task}\n"
    "Before writing the final post, think step-by-step:\n"
    "1. Identify the main safety hazards in a swimming pool.\n"
    "2. Define the necessary behavioral rules to mitigate these hazards.\n"
    "3. Conclude with the physical benefits of swimming in a pragmatic way.\n"
    "Output your reasoning steps first, then write the final blog post."
)
response_cot = run_prompt(cot_prompt)
print(response_cot)


--- 3. Chain-of-Thought (CoT) ---
 Reasoning Steps:

1. Identify the main safety hazards in a swimming pool:
   a. Drowning: The most significant risk associated with swimming pools.
   b. Slip and fall accidents: Due to wet surfaces and slippery pool decks.
   c. Electrical hazards: Pool lights, pumps, and other electrical equipment.
   d. Chemical exposure: Chlorine and other pool chemicals can cause skin and eye irritation.
   e. Overcrowding: Increases the risk of accidents and reduces the effectiveness of supervision.

2. Define the necessary behavioral rules to mitigate these hazards:
   a. Always swim with a buddy or under the supervision of a lifeguard.
   b. Follow pool rules and regulations, including no running, diving in shallow water, and no horseplay.
   c. Use appropriate swimwear and avoid loose clothing that can cause entanglement.
   d. Keep the pool area dry and clean to prevent slips and falls.
   e. Avoid direct contact with pool chemicals and report any spills or

In [ ]:
# ---------------------------------------------------------
# TECHNIQUE 4: Prompt Chaining
# Breaking the task into two real calls
# ---------------------------------------------------------
print("\n--- 4. Prompt Chaining ---")

# Step 1: Get the rules
print("Generating rules...")
prompt_step_1 = "List 3 fundamental safety rules for a public swimming pool. Be brief."
rules_output = run_prompt(prompt_step_1)
print(f"-> Rules Found: {rules_output}\n")

# Step 2: Write the post using those rules
print("Generating final post...")
prompt_step_2 = (
    f"Use the following safety rules to write an introductory blog post about swimming:\n"
    f"{rules_output}\n"
    "The post must instruct citizens to swim while strictly respecting these norms."
)
response_chain = run_prompt(prompt_step_2)
print(response_chain)


--- 4. Prompt Chaining ---
Generating rules...
-> Rules Found:  1. No diving in shallow water.

2. Maintain constant supervision of children.

3. Follow posted depth guidelines and swim within designated areas.

Generating final post...
 Welcome to our community's guide on safe swimming practices. Adherence to the following safety rules is imperative for the well-being of all swimmers:

1. Diving in shallow water is strictly prohibited. This rule is in place to prevent serious injuries that can occur from inadvertent head or spinal trauma.

2. Constant supervision of children is required. It is the responsibility of guardians to ensure the safety of their young ones at all times while in or near water.

3. Swimmers must adhere to posted depth guidelines and confine their activities to designated areas. These measures are designed to minimize the risk of accidents and ensure a safe swimming environment for everyone.

We urge all citizens to respect these regulations. By doing so, we co

In [ ]:
# ---------------------------------------------------------
# TECHNIQUE 5: Tree-of-Thought (ToT)
# Exploring alternatives
# ---------------------------------------------------------
print("\n--- 5. Tree-of-Thought (ToT) ---")
tot_prompt = (
    "You need to write a blog post about swimming. \n"
    "1. Generate 3 different angles/approaches for the post (e.g., Safety Focus, Health Focus, Regulation Focus).\n"
    "2. Evaluate which approach is most suitable for a serious Finnish police officer.\n"
    "3. Select the best option and write the full post based on that choice."
)
response_tot = run_prompt(tot_prompt)
print(response_tot)


--- 5. Tree-of-Thought (ToT) ---
 Approaches for the blog post:

1. Safety Focus: Emphasizing the importance of following safety protocols and guidelines while swimming to prevent accidents and injuries.
2. Health Focus: Highlighting the physical and mental health benefits of swimming, including cardiovascular fitness, muscle strength, and stress reduction.
3. Regulation Focus: Discussing the legal and regulatory aspects of swimming, such as pool rules, lifeguard responsibilities, and compliance with local ordinances.

Evaluation:

As a stereotypical Finnish police officer, the most suitable approach would be the Regulation Focus. This angle aligns with the officer's commitment to order, safety, and adherence to rules. It also provides practical information that can be applied to ensure a safe and lawful swimming environment.

Full blog post (Regulation Focus):

Title: Upholding Safety and Order: The Importance of Regulation in Swimming

Swimming is a popular recreational activity enj